# 标准流
`sys`模块提供了*Python*的标准输入，输出和错误流

In [1]:
import sys
sys.stdin, sys.stdout, sys.stderr

(<_io.TextIOWrapper name='<stdin>' mode='r' encoding='UTF-8'>,
 <ipykernel.iostream.OutStream at 0x7f735362a0f0>)

标准流是预先打开的*Python* **文件对象**。`print()`和`input()`实际上只是标准输出流和输入流的接口，因此它们和`sys.stdin()`/`sys.stdout()`类似：

In [3]:
print('Hello, world!')
sys.stdout.write('Hello, world!' + '\n')

Hello, world!
Hello, world!


In [11]:
input('Your name >>> ')
print('Your name >>> ');sys.stdin.readline()[:-1]

Your name >>> Tom
Your name >>> 


''

## 重定向流到文件或程序
### 重定向的常见作用
- 通过重定向标准输入流到不同的程序，可以将一个测试脚本应用于任意的输出
- 重定向标准输出流使得我们能够保存及后续分析一个程序的输出

示例：teststreams.py

In [5]:
"read numbers till eof and show squares"


def interact():
    "与用户交互计算平方"
    print('Hello, stream world!')
    while True:
        try: reply_str = input('Enter a number >>> ')
        except EOFError: break
        else:
            if reply_str == 'exit': break
            reply_int = int(reply_str)
            print('%s squared is %d' % (reply_str, reply_int ** 2))
    print(':) Bye')
    

if __name__ == '__main__':
    interact()

Hello, stream world!
Enter a number >>> 11
11 squared is 121
Enter a number >>> 6
6 squared is 36
Enter a number >>> exit
:) Bye


- 它从标准输出流中读入数字，直到读入文件结束符（在*Windows*下是`CTRL + Z`，在*Unix*下是`CTRL + D`）
- *EOFError*: 读取到文件结束符
```shell
beacherhou@alone-Vostro-14-5401:/media/beacherhou/Coding/Python/Python项目/pp4e/system$ cat input.txt
11
6
beacherhou@alone-Vostro-14-5401:/media/beacherhou/Coding/Python/Python项目/pp4e/system$ python test_streams.py < input.txt 
Hello, stream world!
Enter a number >>> 11 squared is 121
Enter a number >>> 6 squared is 36
Enter a number >>> :) Bye
```
- 可以利用*shell*语法`< filename`把标准输入流重定向到文件输入
```shell
beacherhou@alone-Vostro-14-5401:/media/beacherhou/Coding/Python/Python项目/pp4e/system$ python test_streams.py < input.txt > output.txt
beacherhou@alone-Vostro-14-5401:/media/beacherhou/Coding/Python/Python项目/pp4e/system$ cat output.txt 
Hello, stream world!
Enter a number >>> 11 squared is 121
Enter a number >>> 6 squared is 36
Enter a number >>> :) Bye
```
- 也可以利用*shell*语法将标准输出流重定向到文件中

### 用管道（*pipe*）链接程序

在两个命令之间使用*shell*字符`|`，可以将一个程序的标准输出发送到另一个程序的标准输入。
```shell
beacherhou@alone-Vostro-14-5401:/media/beacherhou/Coding/Python/Python项目/pp4e/system$ python test_streams.py < input.txt | more
Hello, stream world!
Enter a number >>> 11 squared is 121
Enter a number >>> 6 squared is 36
Enter a number >>> :) Bye
```
使用`stdin`和`stdout`实现脚本程序的通信，可以简化复用过程

示例：sorter.py

In [ ]:
import sys


lines_list = sys.stdin.readlines()
lines_list.sort()
for line_str in lines_list: print(line_str, end='')

示例：adder.py

In [ ]:
sum_int = 0

while True:
    try: data_str = input()
    except EOFError: break
    else:
        sum_int += int(data_str)

print(sum_int)

```shell
beacherhou@alone-Vostro-14-5401:/media/beacherhou/Coding/Python/Python项目/pp4e/system$ cat data.txt 
2007
0000
1106
1234
beacherhou@alone-Vostro-14-5401:/media/beacherhou/Coding/Python/Python项目/pp4e/system$ python sorter.py < data.txt 
0000
1106
1234
2007
beacherhou@alone-Vostro-14-5401:/media/beacherhou/Coding/Python/Python项目/pp4e/system$ python adder.py < data.txt 
4347
beacherhou@alone-Vostro-14-5401:/media/beacherhou/Coding/Python/Python项目/pp4e/system$ cat data.txt | python sorter.py | python adder.py 
4347
```

### adders和sorters的替代编码之选

- sorter.py使用`readlines()`一次性从`stdin`读取所有输出
- adder.py每次只读取一行

一些平台会并行启动用管道链接的程序，在这种系统之下，如果数据量过大，按行读取将会更加优雅

示例：adder2.py

In [ ]:
import sys


sum_int = 0
while True:
	line_str = sys.stdin.readline()
	if not line_str: break
	sum_int += int(line_str)
print(sum_int)

- `int()`可以接受两端带有空白的数字

我们可以利用*Python*最新的*file iterator*来实现它，在file对象上迭代，`for`循环每次自动抓取一行

示例：adder3.py

In [ ]:
import sys


sum_int = 0
for line_str in sys.stdin: sum_int += int(line_str)
print(sum_int)

在*Python 2.4*以后的脚本还可以更精简：使用内部的`sorted()`函数生成器表达式、文件迭代器。

示例：sorter_small.py

In [ ]:
import sys


for line_str in sorted(sys.stdin): print(line_str, end='')

示例：adder_small.py

In [ ]:
import sys


print(sum(int(line_str) for line_str in sys.stdin))

### 重定向流和用户交互

之前，我们曾用*Python*实现了`more`分页程序。

然而，more.py还隐藏着**不易察觉的缺点**：如果使用管道**重定向标准输出流到more**，且输出**足够长**导致more**产生分页**，向用户提示是否分页，脚本会**直接报错EOFError**

原因在于more.py**错误地使用`stdin`**：

- 一方面，它通过调用`input`从`stdin`读取用户的标准输入流

- 另一方面，它又从`stdin`中读取输入文本

当`stdin`流被**重定向到文件或者管道时**，我们无法再用它读取用户输入，此时它**只能获取输入源的文本**。而且，`stdin`在**程序启动前**就被重定向，因此**无法获取重定向前的`stdout`**。

因此，就需要特殊的接口**从键盘**而非`stdin`直接读取用户输入。

- 在*Windows*下，提供了*Python*标准库`msvcrt`

- 在*Unix*下，可以读取设备文件`/dev/tty`

这里给出*Windows*平台下的示例

示例：more_plus.py

In [ ]:
#!/usr/bin/env python
"""
分隔字符串或文本文件并交互的进行分页
"""


def get_reply():
	"读取用户交互式的回复键，即使stdin重定向到某个文件或者管道"
	import sys


	if sys.stdin.isatty():
		return input('More? ')
	else:
		if sys.platform[:3] == 'win':
			import msvcrt


			msvcrt.putch(b'More? ')
			key = msvcrt.getche()
			msvcrt.putch(b'\n')
			return key
		else:
			assert False, 'platform not supported'


def more(text_str, num_lines=15):
	lines = text_str.splitlines()
	while lines:
		chunk = lines[: num_lines]
		lines = lines[num_lines: ]
		for line in chunk: print(line)
		if lines and get_reply() not in [b'y', b'Y']: break


if __name__ == '__main__':
	import sys


	if len(sys.argv) == 1: more(sys.stdin.read())
	else: more(open(sys.argv[1]).read())

回顾一下，我们总共用了4种方式使用more
1. 加载和函数调用
2. 命令行参数传递文件名
3. 重定向`stdin`到文件
4. 通过管道将内容输出到`stdin`

## 重定向流到*Python*对象

在*Python*里，任何**在方法上与文件类似**的对象都可以充当标准流。它**和对象的数据类型无关**，而取决于接口（有时被称为协议）。

- 任何提供了类似于文件`read`方法的对象都可以指定给`sys.stdin`，以从该对象的`read`方法读取输入

- 任何定义了类似于文件`write`方法的对象都可以指定给`sys.stdout`，所有标准输出都将发送到该对象方法上